In [12]:
'''

整體功能描述

'''

'\n\n整體功能描述\n\n'

In [13]:
'''

Application 主架構

'''

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("./line_secret_key",'r'))  # 載入line_secret_key資訊
server_url=secretFileContentJson.get("server_url")  # 讀取webhooks網址

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/Users/USER/PycharmProjects/YY_germination/YY_germination-master/line/YY",   # url連結名稱(https://ngrok/static_url_path/)
            static_folder = "/Users/USER/PycharmProjects/YY_germination/YY_germination-master/line/YY")  #  https://ngrok/static_url_path/對應到的locla資料夾位置
# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))  # 讀取line channel_access_token
handler = WebhookHandler(secretFileContentJson.get("secret_key"))  # 讀取line secret_key

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [14]:
'''

消息判斷器

讀取指定的json檔案後，把json解析成不同格式的SendMessage

讀取檔案，
把內容轉換成json
將json轉換成消息
放回array中，並把array傳出。

'''

# 引用會用到的套件
from linebot.models import (
    ImagemapSendMessage,TextSendMessage,ImageSendMessage,LocationSendMessage,FlexSendMessage
)

from linebot.models.template import (
    ButtonsTemplate,CarouselTemplate,ConfirmTemplate,ImageCarouselTemplate
    
)

from linebot.models.template import *
import sys

def detect_json_array_to_new_message_array(fileName):
    
    #開啟檔案，轉成json
    with open(fileName,encoding="utf8") as f:  # 在windows環境下執行須進行編碼才可讀取中文字 [encoding="utf8"]
        jsonArray = json.load(f)
    
    # 解析json
    returnArray = []
    for jsonObject in jsonArray:

        # 讀取其用來判斷的元件
        message_type = jsonObject.get('type')
        
        # 轉換，判斷讀取的json檔是哪一種line格式
        if message_type == 'text':
            returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'imagemap':
            returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'template':
            returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'image':
            returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'sticker':
            returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'audio':
            returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'location':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'flex':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))    

    # 回傳
    return returnArray

In [15]:
'''

設定訂閱頻道後顯示的圖文
製作文字與圖片的教學訊息

'''
# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 消息清單
reply_message_list = [
    TextSendMessage(text="您好！歡迎使用研耘科技發芽率辨識平台")
]


In [16]:
'''

撰寫用戶關注時，我們要處理的商業邏輯

1. 取得用戶個資，並存回伺服器
2. 把先前製作好的自定義菜單，與用戶做綁定
3. 回應用戶，歡迎用的文字消息與圖片消息

'''


# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

# 載入requests套件
import requests


# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    print(event)
    print(event.reply_token)
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內(後可改為存入DB)
    with open("users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\r\n')
        
        
    # 將菜單綁定在用戶身上
    linkRichMenuId=secretFileContentJson.get("rich_menu_id")  # 載入line_secret_key內的rich_menu_id資訊
    linkMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu/%s' % (event.source.user_id, linkRichMenuId)  # 指定user_id檔定rich_menu_id
    linkMenuRequestHeader={'Content-Type':'image/png','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}  # request header資訊(含channel_access_token)
    lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)  # 致行榜定圖文選單的命令
    
    # 回覆文字消息與圖片消息(綁定圖文選單與回文是不同的指令喔~~)
    line_bot_api.reply_message(
        event.reply_token,
        reply_message_list
    )

In [17]:
'''

handler處理文字消息

收到用戶回應的文字消息，
按文字消息內容，往素材資料夾中，找尋以該內容命名的資料夾，讀取裡面的reply.json

轉譯json後，將消息回傳給用戶

'''
import time
# 引用套件
from linebot.models import (
    MessageEvent, TextMessage
)

# 文字消息處理
@handler.add(MessageEvent,message=TextMessage)
def process_text_message(event):
    if gerData['sawDate'] != None and gerData['done'] == 0:  # 收到品號資訊，再次請對方確認
        if len(event.message.text) == 10:
            result_message_array =[]
            replyJsonPath = "./YY/ger4.json"
            result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
            line_bot_api.reply_message(event.reply_token, result_message_array)
            gerData['id'] = event.message.text    
            gerData['done'] = 1
        else:
            print('error')
        print(gerData)
    else:
        print(event.message.text)

In [18]:
from linebot.models import QuickReply, QuickReplyButton, MessageAction
text_message = ImageSendMessage(
                                original_content_url='https://img14.360buyimg.com/n1/jfs/t13783/319/1236665218/301479/1cc634fe/5a1d2ba2N238bb222.jpg',
                                preview_image_url='https://img14.360buyimg.com/n1/jfs/t13783/319/1236665218/301479/1cc634fe/5a1d2ba2N238bb222.jpg',
                                quick_reply=QuickReply(items=[
                                   QuickReplyButton(action=MessageAction(label="發芽", text="發芽", data='human_check_yes')),
                                   QuickReplyButton(action=MessageAction(label="未發芽", text="未發芽", data='human_check_no'))
                               ]))

# QuickReplyButton(action=action

In [19]:
'''

handler處理Postback Event

載入功能選單與啟動特殊功能

解析postback的data，並按照data欄位判斷處理

'''
from linebot.models import (
    PostbackEvent
)

from urllib.parse import parse_qs 

@handler.add(PostbackEvent)
def process_postback_event(event):
    dict1 = parse_qs(event.postback.data)
    if 'start' == list(dict1.keys())[0]:  # 第一層動作判斷(start)
        func = list(dict1.values())[0][0]  # 判斷是哪一個命令
        result_message_array =[]
        replyJsonPath = './YY/' + func + '.json'  # 回應檔路徑
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)  # 呼叫回應檔
        # 讀資料庫改json
        # ger: 品種名稱、id、縮圖，點more要再列出其他品種
        # human: 品種名稱、id、縮圖，點more要再列出其他品種
        # table: 品種名稱、id、縮圖，點more要再列出其他品種
        # power: 列出現有人員清單
        # problem、add: 文字方框而已，用dict紀錄啟動該功能
        line_bot_api.reply_message(event.reply_token, result_message_array)  # post回應
        
    if 'ger' in list(dict1.keys())[0]:  # 發芽率辨識
        print(dict1.values())
        dict2 = parse_qs(list(dict1.values())[0][0])
        print(dict2)
        if 'getdate' in list(dict2.keys())[0]:  # 確認辨識的作物品種，回傳播種日期選擇
            gerData['name'] = list(dict2.values())[0][0]  # 紀錄是哪一個作物
            result_message_array =[]
            replyJsonPath = './YY/ger2.json'  # 回應檔路徑
            result_message_array = detect_json_array_to_new_message_array(replyJsonPath)  # 呼叫回應檔
            line_bot_api.reply_message(event.reply_token, result_message_array)  # post回應
        if 'selectDate' in list(dict2.keys())[0]:  # 確認播種日期，請輸入品號
            gerData['sawDate'] = event.postback.params['date']
            gerData['done'] = 0
            print(gerData)
            result_message_array =[]
            replyJsonPath = './YY/ger3.json'  # 回應檔路徑
            result_message_array = detect_json_array_to_new_message_array(replyJsonPath)  # 呼叫回應檔
            line_bot_api.reply_message(event.reply_token, result_message_array)  # post回應
        if 'confirm' in list(dict2.keys())[0]:  # 確認作物品種、播種日期、品號無誤後，抓資料庫參數、啟動local設備，清空暫存檔
            # 抓辨識用參數
            # 啟動攝影機、補光燈
            result_message_array =[]
            replyJsonPath = './YY/ger5.json'  # 回應檔路徑
            result_message_array = detect_json_array_to_new_message_array(replyJsonPath)  # 呼叫回應檔
            line_bot_api.reply_message(event.reply_token, result_message_array)  # post回應
            gerData['name'] = None
            gerData['sawDate'] = None
            gerData['id'] = None
            gerData['done'] = None
        if 'shot' in list(dict2.keys())[0]:  # 拍攝照片、判斷、紀錄
            # 拍照
            # 模型判斷
            # 回存資料庫
            result_message_array =[]
            replyJsonPath = './YY/ger6.json'  # 回應檔路徑
            result_message_array = detect_json_array_to_new_message_array(replyJsonPath)  # 呼叫回應檔
            # 回傳辨識結果(文字、圖像)
            line_bot_api.reply_message(event.reply_token, result_message_array)  # post回應
        if 'stop' in list(dict2.keys())[0]:  # 確認停止辨識，統整這一批的辨識結果
            # 抓取這一批的所有辨識紀錄
            result_message_array =[]
            replyJsonPath = './YY/ger7.json'  # 回應檔路徑
            result_message_array = detect_json_array_to_new_message_array(replyJsonPath)  # 呼叫回應檔
            # 回傳整體辨識結果(文字)
            line_bot_api.reply_message(event.reply_token, result_message_array)  # post回應
    if 'human' in list(dict1.keys())[0]:  # 人工判別
        print(dict1.values())
        dict2 = parse_qs(list(dict1.values())[0][0])
        print(dict2)
        if 'species' in list(dict2.keys())[0]:  # 確認判別的品種
            humanData['name'] = list(dict2.values())[0][0]
            humanData['date'] = '2020/3/7'
            # 抓所有待判斷照片出來
            # 回傳一張照片的URL
            replyJsonPath = './YY/human2.json'  # 回應檔路徑
            result_message_array = detect_json_array_to_new_message_array(replyJsonPath)  # 呼叫回應檔
            # 回傳整體辨識結果(文字)
            line_bot_api.reply_message(event.reply_token, result_message_array)  # post回應
            
        if 'check' in list(dict2.keys())[0] and humanData['done'] == 9:  # 判斷是否停止
            print(list(dict2.values())[0][0])
            print(humanData['done'], 'done')
            replyJsonPath = './YY/human4.json'  # 回應檔路徑
            result_message_array = detect_json_array_to_new_message_array(replyJsonPath)  # 呼叫回應檔
            # 回傳整體辨識結果(文字)
            line_bot_api.reply_message(event.reply_token, result_message_array)  # post回應
            
        if 'check' in list(dict2.keys())[0] and humanData['done'] < 9:  # 確認判斷的數量
            humanData['done'] = humanData['done'] + 1
            print(humanData['done'])
            print(list(dict2.values())[0][0])
            replyJsonPath = './YY/human2.json'  # 回應檔路徑
            result_message_array = detect_json_array_to_new_message_array(replyJsonPath)  # 呼叫回應檔
            # 回傳整體辨識結果(文字)
            line_bot_api.reply_message(event.reply_token, result_message_array)  # post回應
        
        if 'continue' in list(dict2.keys())[0]:
            if 'yes' in list(dict2.values())[0][0]:  # 繼續在辨識十張照片
                humanData['done'] = 0
                replyJsonPath = './YY/human2.json'  # 回應檔路徑
                result_message_array = detect_json_array_to_new_message_array(replyJsonPath)  # 呼叫回應檔
                # 回傳整體辨識結果(文字)
                line_bot_api.reply_message(event.reply_token, result_message_array)  # post回應
                
            if 'no' in list(dict2.values())[0][0]:  # 停止辨識並儲存
                humanData['done'] = 0
                replyJsonPath = './YY/human5.json'  # 回應檔路徑
                result_message_array = detect_json_array_to_new_message_array(replyJsonPath)  # 呼叫回應檔
                # 回傳整體辨識結果(文字)
                line_bot_api.reply_message(event.reply_token, result_message_array)  # post回應

In [20]:
gerData = {'name':None, 'sawDate':None, 'id':None, 'done': None}  # 作為執行發芽率辨識的暫存物件
humanData = {'name':None, 'date': None, 'image':None, 'done':0}  # 作為執行人工判別的暫存物件0-10
# addData
# tableData
# powerData
# problemData

In [ ]:
'''

Application 運行（開發版）

'''
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Mar/2020 09:33:08] "POST / HTTP/1.1" 200 -


{"mode": "active", "replyToken": "8260e6d7613e425f993e1cf1437a8767", "source": {"type": "user", "userId": "Ua88d33cc15e1f629a54b415df99c2d62"}, "timestamp": 1584149592393, "type": "follow"}
8260e6d7613e425f993e1cf1437a8767


127.0.0.1 - - [14/Mar/2020 09:33:13] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 09:38:33] "GET /Users/USER/PycharmProjects/YY_germination/YY_germination-master/line/YY/aaa.jpg HTTP/1.1" 404 -
127.0.0.1 - - [14/Mar/2020 09:38:39] "GET /Users/USER/PycharmProjects/YY_germination/YY_germination-master/line/YY/aaa.jpg HTTP/1.1" 404 -
127.0.0.1 - - [14/Mar/2020 09:38:44] "GET /Users/USER/PycharmProjects/YY_germination/YY_germination-master/line/YY/aaa.jpg HTTP/1.1" 404 -
127.0.0.1 - - [14/Mar/2020 09:43:24] "GET /Users/USER/PycharmProjects/YY_germination/YY_germination-master/line/YY/aaa.jpg HTTP/1.1" 404 -
127.0.0.1 - - [14/Mar/2020 10:47:12] "GET /Users/USER/PycharmProjects/YY_germination/YY_germination-master/line/YY/aaa.jpg HTTP/1.1" 404 -
127.0.0.1 - - [14/Mar/2020 10:47:37] "GET /Users/USER/PycharmProjects/YY_germination/YY_germination-master/line/YY/aaa.jpg HTTP/1.1" 404 -
127.0.0.1 - - [14/Mar/2020 10:47:40] "GET /Users/USER/PycharmProjects/YY_germination/YY_germination-maste

{"mode": "active", "replyToken": "f620237307604d16ade686baf627acfb", "source": {"type": "user", "userId": "Ubbff11f7cb4b9a9c3d63ed602e365acf"}, "timestamp": 1584154077487, "type": "follow"}
f620237307604d16ade686baf627acfb


...

人工判別


127.0.0.1 - - [14/Mar/2020 11:51:45] "POST / HTTP/1.1" 200 -


dict_values([['species=3G1']])
{'species': ['3G1']}


127.0.0.1 - - [14/Mar/2020 11:51:52] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 11:51:52] "POST / HTTP/1.1" 200 -


判斷芝麻葉(3G1)


In [ ]:
'''

Application 運行（heroku版）

'''

# import os
# if __name__ == "__main__":
#     app.run(host='0.0.0.0',port=os.environ['PORT'])

# 工作進度：
* 2020/3/2
    - 完成後台骨架與json模板
    - 完成發芽率辨識1-3
    
* 2020/3/5
    - 修改發芽率辨識模組(都用postback接收指令)
    - 整理if else架構
    
* 問題
    - 線下圖片檔(圖檔或代碼型態)如何讀取(已解決圖檔問題，代碼格式尚未測試)
    - 每個動作紀錄於暫存檔時需紀錄操作者id，不可混淆不同人的操作歷程
    
    
    